# W&B Production Monitoring Overview
In this notebooks you will learn to monitor your production models with W&B through an illustrative example. We will train a model to correctly identify handwritten digits, then monitor a locally deployed version of the model.

_Note: To keep the example focused on important code, much of the dataset manipulation, modelling, and other utilities are packaged in local files and imported here_

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## Step 1: Get Data
In this example, we will use `keras.datasets.mnist.load_data()` to load in the MNIST dataset. 

In [ ]:
import model_util

dataset = model_util.get_dataset()
model_util.image_from_array(dataset[0][0])

## Step 2: Train Model
Next we will train a classic NN to predict the digits

In [ ]:
model = model_util.train_model(*dataset, conv_layers=0, epochs=1) # 1 epoch so we can actually see some errors

# Step 3: Query Model
Now, let's query the model! Normally there is a little pre- and post- processing needed to make a prediction - we will write a short function to handle this for us.

In [ ]:
import numpy as np
import json

def simple_predict(image_arr):
    # Prepare image for model
    tensor = (image_arr.astype("float32")).reshape(1, 28, 28, 1)

    # Make the prediction
    prediction = model.predict(tensor, verbose=False)

    # In this application, we need to reshape the output:
    raw_predictions = prediction[0].tolist()
    logits = {
        str(k): v for k, v in zip(range(10), raw_predictions)
    }
    
    prediction = np.argmax(raw_predictions).tolist()
    
    return {"logits": logits, "prediction": prediction}

_, _, x_test, y_test = dataset
for i in range(10):
    image_arr = x_test[i]
    truth = y_test[i]
    preds = simple_predict(image_arr)
    
    print(f"Input: {truth}")
    display(model_util.image_from_array(image_arr))
    print(f"Prediction: {preds['prediction']}")
    print(f"Logits: {json.dumps(preds['logits'], indent=2)}")
    print("")
    

# Step 3a: Save Predictions with W&B Weave using StreamTable
With W&B's Weave library, we can stream any data to W&B for storage and further analysis.

In [ ]:
# Only run this if you are running Weave locally (as opposed to installing from Pypy)
import weave
weave.use_frontend_devmode()

In [ ]:
from weave.monitoring import StreamTable

# Initialize a stream table
st = StreamTable("timssweeney/demos/mnist_monitoring_example_4")
_, _, x_test, y_test = dataset
for i in range(100):
    image_arr = x_test[i]
    truth = y_test[i].tolist()
    preds = simple_predict(image_arr)
    
    # Log the data
    st.log({
        **preds,
        "image": model_util.image_from_array(image_arr),
        "truth": truth
    })

# Optional: wait for the logs to finish uploading (nicer for live demos)
st.finish()

# Show the StreamTable
st    

# Step 3a: Save Predictions with W&B Weave using `monitor` decorator
The pattern of logging inputs and outputs of a functions is so common, that we provide a decorator which automatically logs the functions i/o.

In [ ]:
from weave.monitoring import monitor
import numpy as np

@monitor(
    entity_name="timssweeney", project_name="prodmon",
    # Setting `auto_log=False` will return a PredictionRecord which 
    # supports additional data being added (in this case the ground truth)
    auto_log = False,
    # An preprocessor allows the function arguments to be pre-processed before logging.
    input_preprocessor = lambda image_arr: {"image": model_util.image_from_array(image_arr)}
)
def monitor_predict(image_arr):
    # Prepare image for model
    tensor = (image_arr.astype("float32")).reshape(1, 28, 28, 1)

    # Make the prediction
    prediction = model.predict(tensor, verbose=False)

    # In this application, we need to reshape the output:
    raw_predictions = prediction[0].tolist()
    logits = {
        str(k): v for k, v in zip(range(10), raw_predictions)
    }
    
    prediction = np.argmax(raw_predictions).tolist()
    
    return {"logits": logits, "prediction": prediction}

_, _, x_test, y_test = dataset
for i in range(100):
    image_arr = x_test[i]
    truth = y_test[i].tolist()
    preds = monitor_predict(image_arr)
    
    # Now the output of `monitored_predict` is a `PredictionRecord`
    preds.add_data({"truth": truth})
    preds.finalize()

# Step 4: End to End Example
Typically a production application will contain a prediction service that provides predictions to a client. To demonstrate this in a notebook, we will create a `PredictionService` and an `AppUI` (a small interface to allow the user to draw an image, view the prediction, and give feedback on results) that communicate via `predict` and `record_feedback` methods. 

_Note: this is just for exmaple purposes, your production systems may widely vary in structure_

In [ ]:
import app_util
from weave.monitoring import monitor
import PIL
import numpy as np

class PredictionService(app_util.PredictionServiceInterface):
    def __init__(self, model):
        self.model = model
        self.last_prediction = {}
    
    @monitor(auto_log = False,  entity_name="timssweeney", project_name="prodmon")
    def _raw_predict(self, pil_image: PIL.Image) -> dict:
        # Prepare image for model
        tensor = (np.array(pil_image.resize((28, 28))).astype("float32") / 255).reshape(1, 28, 28, 1)

        # Make the prediction
        prediction = self.model.predict(tensor, verbose=False)

        # In this application, we need to reshape the output:
        raw_predictions = prediction[0].tolist()
        logits = {
            str(k): v for k, v in zip(range(10), raw_predictions)
        }

        prediction = np.argmax(raw_predictions).tolist()

        return {"logits": logits, "prediction": prediction}
    
    def _update_last_prediction(self, prediction) -> None:
        if len(self.last_prediction) > 0:
            last_pred = self.last_prediction.pop(list(self.last_prediction.keys())[0])
            last_pred.finalize()
        self.last_prediction[prediction.id] = prediction

    
    def predict(self, pil_image: PIL.Image) -> app_util.Prediction:
        record = self._raw_predict(pil_image)
        
        # Cache the last prediction for ground_truth recording
        self._update_last_prediction(record)
        
        # Return the prediction
        return app_util.Prediction(record.get()['logits'], record.id)
    
    def record_feedback(self, prediction_id: str, feedback: int) -> None:
        if prediction_id not in self.last_prediction:
            return

        # Get the past prediction
        prediction = self.last_prediction.pop(prediction_id)
        
        # Save the user feedback
        prediction.add_data({'user_feedback': feedback})
        
        # Log the results
        prediction.finalize()
        
app_util.render_app(PredictionService(model))